In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import time
from tensorflow.keras.optimizers import Adam


In [2]:
# Ensure reproducibility
tf.random.set_seed(42)
np.random.seed(42)


In [3]:
import os  # Add this line at the beginning of your code

save_dir = '/kaggle/working/'  # Specify your directory path
os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist


In [4]:
# Batch size and learning rate
batch_size = 16  
epochs = 50
latent_dim = 128

# Optimizers
opt_autoencoder = Adam(learning_rate=0.0001, beta_1=0.5)
opt_disc = Adam(learning_rate=0.0001, beta_1=0.5)


In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import os
from sklearn.model_selection import train_test_split

# Paths to the dataset
image_dir_1 = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1'  
image_dir_2 = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2'  
metadata_file = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv'  


# Load metadata CSV
labels = pd.read_csv(metadata_file)

# Convert diagnosis (dx) to integers (you can map specific diseases if needed)
label_mapping = {label: idx for idx, label in enumerate(labels['dx'].unique())}
labels['dx'] = labels['dx'].map(label_mapping)

# Set image size for the model
IMG_SIZE = 64  # Resize images to 64x64

# Function to load and preprocess images
def load_images_and_labels(labels, image_dir_1, image_dir_2, img_size=64):
    images = []
    label_list = []

    for _, row in labels.iterrows():
        # Get the file path for the image
        file_path_1 = os.path.join(image_dir_1, row['image_id'] + '.jpg')
        file_path_2 = os.path.join(image_dir_2, row['image_id'] + '.jpg')

        # Check if the image exists in part 1 or part 2
        if os.path.exists(file_path_1):
            img = image.load_img(file_path_1, target_size=(img_size, img_size))
        elif os.path.exists(file_path_2):
            img = image.load_img(file_path_2, target_size=(img_size, img_size))
        else:
            continue  # Skip if image file is missing

        # Convert image to numpy array and normalize it
        img_array = image.img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
        images.append(img_array)
        label_list.append(row['dx'])  # Add the label (dx)

    X = np.array(images)  # Shape: (num_samples, img_size, img_size, 3)
    y = np.array(label_list)  # Labels: [0, 1, 2, ..., 6] for classification
    return X, y

# Load the images and labels
X, y = load_images_and_labels(labels, image_dir_1, image_dir_2, IMG_SIZE)

# Step 1: Split into 80% training and 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Split the 80% training set into 80% training and 20% validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Check the shape of the dataset
print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)

# Convert labels to one-hot encoding
y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes=7)  # Assuming 7 classes
y_val_one_hot = tf.keras.utils.to_categorical(y_val, num_classes=7)
y_test_one_hot = tf.keras.utils.to_categorical(y_test, num_classes=7)

# Create the data pipeline
def create_dataset(X_train, y_train_one_hot, X_val, y_val_one_hot, X_test, y_test_one_hot, batch_size):
    train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train_one_hot))
    val_data = tf.data.Dataset.from_tensor_slices((X_val, y_val_one_hot))
    test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test_one_hot))

    # Efficient data pipeline: cache, batch, and prefetch
    train_data = (train_data
                  .cache()
                  .shuffle(buffer_size=1000)
                  .batch(batch_size)
                  .prefetch(tf.data.experimental.AUTOTUNE))
    
    val_data = (val_data
                .cache()
                .batch(batch_size)
                .prefetch(tf.data.experimental.AUTOTUNE))

    test_data = (test_data
                 .cache()
                 .batch(batch_size)
                 .prefetch(tf.data.experimental.AUTOTUNE))

    return train_data, val_data, test_data

# Set batch size for training
batch_size = 16

# Create the dataset pipeline
train_data, val_data, test_data = create_dataset(X_train, y_train_one_hot, X_val, y_val_one_hot, X_test, y_test_one_hot, batch_size)


Training set shape: (6409, 64, 64, 3)
Validation set shape: (1603, 64, 64, 3)
Test set shape: (2003, 64, 64, 3)


In [6]:
# Encoder with Dropout and BatchNormalization
encoder = tf.keras.Sequential([
    layers.Input(shape=(64, 64, 3)),
    layers.Conv2D(64, kernel_size=3, strides=2, padding="valid"),
    layers.LeakyReLU(0.2),
    layers.BatchNormalization(),
    layers.Dropout(0.3),  # Add dropout
    layers.Conv2D(128, kernel_size=3, strides=2, padding="valid"),
    layers.LeakyReLU(0.2),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Conv2D(256, kernel_size=3, strides=2, padding="valid"),
    layers.LeakyReLU(0.2),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Flatten(),
    layers.Dense(latent_dim)
])

# Decoder with BatchNormalization and Dropout
decoder = tf.keras.Sequential([
    layers.Input(shape=(latent_dim,)),
    layers.Dense(8 * 8 * 128),
    layers.Reshape((8, 8, 128)),
    layers.Conv2DTranspose(128, kernel_size=3, strides=2, padding="same", activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),  # Add dropout
    layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding="same", activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Conv2DTranspose(32, kernel_size=3, strides=2, padding="same", activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2DTranspose(3, kernel_size=3, padding="same", activation='sigmoid')
])

# Full autoencoder model
autoencoder_input = layers.Input(shape=(64, 64, 3))
latent_representation = encoder(autoencoder_input)
reconstructed_image = decoder(latent_representation)

autoencoder = models.Model(inputs=autoencoder_input, outputs=reconstructed_image)

# Compile the autoencoder model
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse')

# Print model summary to check the architecture
autoencoder.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential (Sequential)              │ (None, 128)                 │       1,978,368 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_1 (Sequential)            │ (None, 64, 64, 3)           │       1,298,371 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,276,739 (12.50 MB)

 Trainable params: 3,275,395 (12.49 MB)

 Non-trainable params: 1,344 (5.25 KB)

In [7]:
from tensorflow.keras import layers, models
import tensorflow as tf
from tensorflow.keras import regularizers

def create_model_with_reg():
    inputs = tf.keras.Input(shape=(64, 64, 3))
    
    # First Convolutional Block with valid padding (no padding) and L2 regularization
    x = tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding='valid',
                               kernel_regularizer=regularizers.l2(0.01))(inputs)  # L2 regularization added
    x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)  # Output shape: (31, 31, 32)
    
    # Second Convolutional Block with valid padding (no padding) and L2 regularization
    x = tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='valid',
                               kernel_regularizer=regularizers.l2(0.01))(x)  # L2 regularization added
    x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)  # Output shape: (14, 14, 64)
   
    # Flatten the feature map
    x = tf.keras.layers.Flatten()(x)

    # Dense Layers with L2 regularization
    x = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)  # L2 regularization added
    x = tf.keras.layers.Dropout(0.3)(x)  # Slightly reduced dropout
    x = tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)  # L2 regularization added
    x = tf.keras.layers.Dropout(0.4)(x)   
    outputs = tf.keras.layers.Dense(7, activation='softmax')(x)  # 7 output classes (for HAM10000 dataset)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    return model

# Create an instance of the model
discriminator = create_model_with_reg()

# Compile the model
discriminator.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary to verify the changes
discriminator.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       1,605,760 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 7)                   │             455 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,633,863 (6.23 MB)

 Trainable params: 1,633,863 (6.23 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
from tensorflow.keras import layers, models, optimizers
import tensorflow as tf

# Loss functions
loss_fn_auto = tf.keras.losses.MeanSquaredError()  # Autoencoder Loss for reconstruction
loss_fn_disc = tf.keras.losses.CategoricalCrossentropy()  # Discriminator Loss for multi-class classification

# Optimizers
opt_autoencoder = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5)  # Optimizer for Autoencoder
opt_disc = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5)  # Optimizer for Discriminator

# Make sure to compile the model before training
autoencoder.compile(optimizer=opt_autoencoder, loss=loss_fn_auto)
discriminator.compile(optimizer=opt_disc, loss=loss_fn_disc, metrics=['accuracy'])


# Print model summaries to check
print("Autoencoder Model Summary:")
autoencoder.summary()

print("\nDiscriminator Model Summary:")
discriminator.summary()


Autoencoder Model Summary:


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential (Sequential)              │ (None, 128)                 │       1,978,368 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_1 (Sequential)            │ (None, 64, 64, 3)           │       1,298,371 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,276,739 (12.50 MB)

 Trainable params: 3,275,395 (12.49 MB)

...

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       1,605,760 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 7)                   │             455 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,633,863 (6.23 MB)

 Trainable params: 1,633,863 (6.23 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
from tqdm import tqdm
# Training loop for GAN
autoencoder_losses_epoch = []
discriminator_losses_epoch = []
discriminator_acc_epoch = []
val_autoencoder_losses_epoch = []
val_discriminator_losses_epoch = []
val_discriminator_acc_epoch = []

for epoch in range(50):
    # Initialize epoch-wise metrics
    epoch_autoencoder_losses = []
    epoch_discriminator_losses = []
    epoch_discriminator_acc = []
    epoch_val_autoencoder_losses = []
    epoch_val_discriminator_losses = []
    epoch_val_discriminator_acc = []

    # Training loop
    for idx, (batch_real, batch_label) in enumerate(tqdm(train_data)):
        with tf.GradientTape() as autoencoder_tape, tf.GradientTape() as disc_tape:
            # Pass batch through the encoder and decoder (autoencoder)
            latent_representation = encoder(batch_real)
            generated_image = decoder(latent_representation)
            loss_autoencoder = loss_fn_auto(batch_real, generated_image)

            # Pass batch through the discriminator (real vs fake image)
            fake = generated_image
            loss_disc_real = loss_fn_disc(batch_label, discriminator(batch_real))
            loss_disc_fake = loss_fn_disc(batch_label, discriminator(fake))
            loss_disc = (loss_disc_real + loss_disc_fake) / 2

        # Apply gradients to autoencoder
        grads_autoencoder = autoencoder_tape.gradient(loss_autoencoder, autoencoder.trainable_weights)
        opt_autoencoder.apply_gradients(zip(grads_autoencoder, autoencoder.trainable_weights))

        # Apply gradients to discriminator
        grads_discriminator = disc_tape.gradient(loss_disc, discriminator.trainable_weights)
        opt_disc.apply_gradients(zip(grads_discriminator, discriminator.trainable_weights))

        # Store batch-wise metrics
        epoch_autoencoder_losses.append(loss_autoencoder.numpy())
        epoch_discriminator_losses.append(loss_disc.numpy())
        epoch_discriminator_acc.append(tf.reduce_mean(
            tf.keras.metrics.categorical_accuracy(batch_label, discriminator(fake))).numpy())
        
        if idx % 200 == 0:
            img = tf.keras.preprocessing.image.array_to_img(fake[0])
            img.save(f"{save_dir}/generated_img_epoch_{epoch}_batch_{idx}.png")

    # Validation loop
    for val_batch_real, val_batch_label in val_data:
        val_latent_representation = encoder(val_batch_real)
        val_generated_image = decoder(val_latent_representation)
        val_loss_autoencoder = loss_fn_auto(val_batch_real, val_generated_image)
        val_loss_disc_real = loss_fn_disc(val_batch_label, discriminator(val_batch_real))
        val_loss_disc_fake = loss_fn_disc(val_batch_label, discriminator(val_generated_image))
        val_loss_disc = (val_loss_disc_real + val_loss_disc_fake) / 2

        # Calculate accuracy
        val_discriminator_accuracy = tf.reduce_mean(
            tf.keras.metrics.categorical_accuracy(val_batch_label, discriminator(val_generated_image)))

        # Store validation metrics
        epoch_val_autoencoder_losses.append(val_loss_autoencoder.numpy())
        epoch_val_discriminator_losses.append(val_loss_disc.numpy())
        epoch_val_discriminator_acc.append(val_discriminator_accuracy.numpy())

    # Calculate epoch-wise averages and append to the lists
    avg_autoencoder_loss = np.mean(epoch_autoencoder_losses)
    avg_discriminator_loss = np.mean(epoch_discriminator_losses)
    avg_discriminator_acc = np.mean(epoch_discriminator_acc)
    avg_val_autoencoder_loss = np.mean(epoch_val_autoencoder_losses)
    avg_val_discriminator_loss = np.mean(epoch_val_discriminator_losses)
    avg_val_discriminator_acc = np.mean(epoch_val_discriminator_acc)

    autoencoder_losses_epoch.append(avg_autoencoder_loss)
    discriminator_losses_epoch.append(avg_discriminator_loss)
    discriminator_acc_epoch.append(avg_discriminator_acc)
    val_autoencoder_losses_epoch.append(avg_val_autoencoder_loss)
    val_discriminator_losses_epoch.append(avg_val_discriminator_loss)
    val_discriminator_acc_epoch.append(avg_val_discriminator_acc)

    # Print epoch-wise metrics
    print(f"Epoch {epoch}, Autoencoder Loss: {avg_autoencoder_loss}, Discriminator Loss: {avg_discriminator_loss}, Discriminator Accuracy: {avg_discriminator_acc}")
    print(f"Validation - Autoencoder Loss: {avg_val_autoencoder_loss}, Discriminator Loss: {avg_val_discriminator_loss}, Discriminator Accuracy: {avg_val_discriminator_acc}")


100%|██████████| 401/401 [05:14<00:00,  1.28it/s]


Epoch 0, Autoencoder Loss: 0.01810292713344097, Discriminator Loss: 1.1084791421890259, Discriminator Accuracy: 0.6721736788749695
Validation - Autoencoder Loss: 0.007117317523807287, Discriminator Loss: 1.0184978246688843, Discriminator Accuracy: 0.6606847643852234


100%|██████████| 401/401 [05:05<00:00,  1.31it/s]


Epoch 1, Autoencoder Loss: 0.004911016207188368, Discriminator Loss: 0.9701564908027649, Discriminator Accuracy: 0.6732301115989685
Validation - Autoencoder Loss: 0.004112974740564823, Discriminator Loss: 0.9650706052780151, Discriminator Accuracy: 0.6600660085678101


100%|██████████| 401/401 [04:59<00:00,  1.34it/s]


...

Epoch 47, Autoencoder Loss: 0.0012854643864557147, Discriminator Loss: 0.3432448208332062, Discriminator Accuracy: 0.8987253904342651
Validation - Autoencoder Loss: 0.0015623506624251604, Discriminator Loss: 0.841713011264801, Discriminator Accuracy: 0.7380362749099731


100%|██████████| 401/401 [04:59<00:00,  1.34it/s]


Epoch 48, Autoencoder Loss: 0.001276260707527399, Discriminator Loss: 0.3277621269226074, Discriminator Accuracy: 0.9063279032707214
Validation - Autoencoder Loss: 0.001431110082194209, Discriminator Loss: 0.8509377837181091, Discriminator Accuracy: 0.7361798286437988


100%|██████████| 401/401 [04:54<00:00,  1.36it/s]


Epoch 49, Autoencoder Loss: 0.001276496914215386, Discriminator Loss: 0.3191652297973633, Discriminator Accuracy: 0.9099127054214478
Validation - Autoencoder Loss: 0.0014815873000770807, Discriminator Loss: 0.9720004200935364, Discriminator Accuracy: 0.7367986440658569
